第4天理解自动微分机制（PyTorch的backward或TF的GradientTape）手动实现简单线性回归

自动微分（Automatic Differentiation，AD）是深度学习框架（如PyTorch和TensorFlow）的核心机制，用于自动计算梯度。

In [2]:
import torch as th
import tensorflow as tf

 # 基本思想
 **链式法则** 复杂函数梯度分解为原子操作的梯度组合。（追踪计算过程实现高效梯度计算）
  * 前向模式 ：计算顺序逐层计算，适合输入维度低
  * 反向模式： 从输出反向逐层累积梯度，适合输出维度低的情况（如损失函数），深度学习框架主要用此模式。

# 2. PyTorch的自动微分（backward）
 PyTorch通过动态计算图（Dynamic Computation Graph）实现自动微分。

###  追踪计算过程：

 * 当张量（Tensor）的requires_grad=True时，所有相关计算会被记录为计算图。

 * 每个操作生成一个Function节点（包含前向和反向的实现）。


In [4]:
import torch
x = torch.tensor(2.0, requires_grad=True)
y = x ** 2 + 3 * x  # 构建计算图

### 反向传播
 * 调用y.backward()时，PyTorch从y开始反向遍历计算图。

 * 对每个节点应用链式法则，计算梯度并存储在.grad属性中。

In [5]:
y.backward()        # 反向传播
print(x.grad)       # 输出梯度：2*2 + 3 = 7.0

tensor(7.)


* 特性：

 * 动态图：每次前向传播都会构建新计算图，支持灵活控制流（如循环、条件）。

 * 梯度累加：默认梯度会累加，需手动清零（optimizer.zero_grad()）。

 * 非标量梯度：对向量输出需传入gradient参数（形状匹配）

# 3 TensorFlow的自动微分（GradientTape）

TensorFlow通过tf.GradientTape上下文管理器记录计算过程，支持动态图和静态图。

###  关键步骤：
 * 记录计算过程：

  * 在GradientTape上下文中执行的操作会被记录


In [6]:
import tensorflow as tf
x = tf.Variable(2.0)
with tf.GradientTape() as tape:
    y = x ** 2 + 3 * x

* 计算梯度：
 * 使用tape.gradient(target, sources)计算梯度。






In [7]:
dy_dx = tape.gradient(y, x)  # 梯度计算
print(dy_dx.numpy())         # 输出：7.0

7.0


* 特性：

  * 默认只记录一次：通过persistent=True可多次计算梯度。

  * 资源释放：非持久模式自动释放资源，避免内存泄漏。

  * Eager Execution：默认启用动态图，与静态图兼容

# 注意事项
 * 内存管理：
  
  反向传播后，PyTorch自动释放计算图；TensorFlow的GradientTape默认只记录一次。

 * 不可变操作：

 某些操作（如+=）可能破坏梯度追踪，需使用x.assign()（TensorFlow）或避免原地操作。

 * 性能优化：

 静态图（如@tf.function）可通过编译提高速度，但牺牲灵活性。

# 手动实现线性回归模型（Pytorch）


In [8]:
# prompt: PyTorch手动实现简单线性回归

import torch

# 1. 准备数据
X = torch.tensor([[1.0], [2.0], [3.0]])  # 特征
y = torch.tensor([[2.0], [4.0], [5.0]])  # 目标变量

# 2. 初始化参数
w = torch.randn(1, requires_grad=True)
b = torch.randn(1, requires_grad=True)

# 3. 设置学习率和迭代次数
learning_rate = 0.01
epochs = 1000

# 4. 训练模型
for epoch in range(epochs):
    # 计算预测值
    y_pred = X * w + b

    # 计算损失
    loss = torch.mean((y_pred - y) ** 2)

    # 反向传播计算梯度
    loss.backward()

    # 更新参数
    with torch.no_grad():
        w -= learning_rate * w.grad
        b -= learning_rate * b.grad
        w.grad.zero_()  # 清除梯度
        b.grad.zero_()  # 清除梯度

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}, w: {w.item():.4f}, b: {b.item():.4f}')

# 5. 打印最终结果
print(f'Final w: {w.item():.4f}, b: {b.item():.4f}')


Epoch [100/1000], Loss: 0.0581, w: 1.4418, b: 0.7990
Epoch [200/1000], Loss: 0.0571, w: 1.4542, b: 0.7707
Epoch [300/1000], Loss: 0.0565, w: 1.4640, b: 0.7484
Epoch [400/1000], Loss: 0.0562, w: 1.4717, b: 0.7309
Epoch [500/1000], Loss: 0.0559, w: 1.4778, b: 0.7172
Epoch [600/1000], Loss: 0.0558, w: 1.4825, b: 0.7064
Epoch [700/1000], Loss: 0.0557, w: 1.4863, b: 0.6979
Epoch [800/1000], Loss: 0.0556, w: 1.4892, b: 0.6912
Epoch [900/1000], Loss: 0.0556, w: 1.4915, b: 0.6860
Epoch [1000/1000], Loss: 0.0556, w: 1.4933, b: 0.6818
Final w: 1.4933, b: 0.6818


# TensorFlow手动实现简单线性回归（输出）

In [9]:
# prompt: TensorFlow手动实现简单线性回归

import tensorflow as tf

# 1. 准备数据
X = tf.constant([[1.0], [2.0], [3.0]], dtype=tf.float32)  # 特征
y = tf.constant([[2.0], [4.0], [5.0]], dtype=tf.float32)  # 目标变量

# 2. 初始化参数
w = tf.Variable(tf.random.normal([1, 1]), dtype=tf.float32)
b = tf.Variable(tf.random.normal([1, 1]), dtype=tf.float32)

# 3. 设置学习率和迭代次数
learning_rate = 0.01
epochs = 1000

# 4. 训练模型
for epoch in range(epochs):
    with tf.GradientTape() as tape:
        # 计算预测值
        y_pred = tf.matmul(X, w) + b

        # 计算损失
        loss = tf.reduce_mean(tf.square(y_pred - y))

    # 计算梯度
    dw, db = tape.gradient(loss, [w, b])

    # 更新参数
    w.assign_sub(learning_rate * dw)
    b.assign_sub(learning_rate * db)

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.numpy():.4f}, w: {w.numpy()[0][0]:.4f}, b: {b.numpy()[0][0]:.4f}')

# 5. 打印最终结果
print(f'Final w: {w.numpy()[0][0]:.4f}, b: {b.numpy()[0][0]:.4f}')


Epoch [100/1000], Loss: 0.0584, w: 1.4381, b: 0.8073
Epoch [200/1000], Loss: 0.0573, w: 1.4514, b: 0.7772
Epoch [300/1000], Loss: 0.0566, w: 1.4618, b: 0.7536
Epoch [400/1000], Loss: 0.0562, w: 1.4699, b: 0.7350
Epoch [500/1000], Loss: 0.0560, w: 1.4764, b: 0.7204
Epoch [600/1000], Loss: 0.0558, w: 1.4814, b: 0.7089
Epoch [700/1000], Loss: 0.0557, w: 1.4854, b: 0.6999
Epoch [800/1000], Loss: 0.0557, w: 1.4885, b: 0.6928
Epoch [900/1000], Loss: 0.0556, w: 1.4910, b: 0.6872
Epoch [1000/1000], Loss: 0.0556, w: 1.4929, b: 0.6828
Final w: 1.4929, b: 0.6828
